In [31]:
import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
import argparse

from dataset_paths import *

warnings.filterwarnings('ignore')

output_version = "20241229"
outdir = "/media/hieunguyen/HNSD_mini/outdir"
dataset_name = "LOD"
mode = "all"
PROJECT = "combine_ctcandi_ichorcna"

input_cancer_class = "Liver"

path_to_main_output = os.path.join(outdir, PROJECT, output_version, dataset_name)
path_to_07_output = os.path.join(path_to_main_output, f"07_output_{mode}", input_cancer_class)
path_to_08_output = os.path.join(path_to_main_output, f"08_output_{mode}", input_cancer_class)
path_to_09_output = os.path.join(path_to_main_output, f"09_output_{mode}", input_cancer_class)
os.system(f"mkdir -p {path_to_09_output}")

convert_ratio = {
        '50' : 0.5, 
        '100': 1, 
        '0.5': 0.005, 
        '25': 0.25, 
        '15': 0.15, 
        '5': 0.05, 
        '1': 0.01, 
        'HC': 0
    }

metadata = pd.read_csv(f"./metadata/{dataset_name}.csv")
metadata["SampleID"] = metadata["Sample"].values
metadata = metadata[metadata["Sample"].duplicated() == False]
metadata["spike_in_ratio"] = metadata["spike-in"].apply(lambda x: convert_ratio[x])
metadata['Label'] = metadata["LABEL"].apply(lambda x: "CRC" if x == "Colorectal cancer" else x.split(" ")[0])
metadata["spike_in_label"] = metadata["Simulated TF"].apply(lambda x: "Control" if x == "Healthy-control" else input_cancer_class)
metadata["Label"] = metadata[["Label", "spike_in_label"]].apply(lambda x: x[0] if x[1] != "Control" else "Control", axis = 1)

if input_cancer_class != "pan_cancer":
    metadata = metadata[metadata["Label"].isin([input_cancer_class, "Control"])]
fulldf = pd.read_csv(f"{path_to_08_output}/feature.csv", index_col = [0])

fulldf = fulldf.merge(metadata[["SampleID", "spike_in_ratio", "Actual tumor_fraction_ichorCNA"]], right_on = "SampleID", left_on = "SampleID")
fulldf.columns = ['SampleID', 'raw_count', 'in_read_count', 'mean_candi_reads',
       'ratio_raw', 'ratio_in_read', 'spike_in_ratio',
       'ichorCNA']

fulldf.to_csv(os.path.join(path_to_09_output, "LOD_dataset_feature_for_training_models.csv"))


In [32]:
fulldf

,SampleID,raw_count,in_read_count,mean_candi_reads,ratio_raw,ratio_in_read,spike_in_ratio,ichorCNA
0,LODLIV12,18031,9085,3.874396,0.000215,0.000426,0.05,0.05390
1,LODCONTROL3,12360,6358,2.530000,0.000205,0.000398,0.00,0.02966
2,LIVER3R9,21510,11133,11.909091,0.000554,0.001070,1.00,0.29280
3,LODLIV15R1,47104,24513,20.884058,0.000443,0.000852,0.50,0.20850
4,CONTROL44R1,29254,15125,5.888889,0.000201,0.000389,0.00,0.01926
...,...,...,...,...,...,...,...,...
127,CONTROL30R3,19151,9282,5.054187,0.000264,0.000545,0.00,0.02547
128,LODLIV3R1,25063,12756,6.798077,0.000271,0.000533,0.25,0.11290
129,LODLIV17R2,28883,15090,7.425837,0.000257,0.000492,0.25,0.05877
130,CONTROL31R2,9162,4580,1.813472,0.000198,0.000396,0.00,0.03182
